In [0]:
import random
import itertools
import numpy as np
import tensorflow as tf
from keras.datasets import cifar10
from keras import backend as K
from keras import Sequential, utils
# from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Conv2D, Dense, MaxPooling2D, Flatten
from keras.optimizers import Adam 
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import SGD
from keras.constraints import maxnorm
from keras.callbacks import EarlyStopping
from keras.preprocessing import image

In [0]:
def create_model():
    weight_decay = 1e-4
    model = Sequential()
    
    
    model.add(Conv2D(filters=64,kernel_size=(3,3),strides=(1,1),padding='same',activation='relu',input_shape=x_train.shape[1:]))
    
    model.add(BatchNormalization())
    
    model.add(Conv2D(filters=64,kernel_size=(3,3),strides=(1,1),padding='same',activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2),padding='valid'))
    
    model.add(Dropout(0.2))

    model.add(Conv2D(filters=128,kernel_size=(3,3),strides=(1,1),padding='same', kernel_regularizer=regularizers.l2(weight_decay),activation='relu'))
    
    model.add(BatchNormalization())
    model.add(Conv2D(filters=128,kernel_size=(3,3),strides=(1,1),padding='same',kernel_regularizer=regularizers.l2(weight_decay),activation='relu'))
   
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))
   
    model.add(Dropout(0.3))

    model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.4))

    model.add(Flatten())
    model.add(Dense(10, activation='softmax'))
    
    return model
    
    

In [0]:
np.random.seed(1)
random.seed(1)
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1,
                              inter_op_parallelism_threads=1)

tf.set_random_seed(1)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

In [0]:

(x_train,y_train),(x_test,y_test) = cifar10.load_data()


In [0]:
y_train = utils.to_categorical(y_train, 10)
y_test = utils.to_categorical(y_test, 10)

In [32]:
# normalization
x_train = (x_train - x_train.min(axis=0)) / (x_train.max(axis=0) - x_train.min(axis=0))
x_test = (x_test - x_test.min(axis=0)) / (x_test.max(axis=0) - x_test.min(axis=0))
x_train.shape

(50000, 32, 32, 3)

In [0]:
#data augmentation part

from keras.preprocessing.image import ImageDataGenerator
x_train2=np.array(x_train,copy=True)
y_train2=np.array(y_train,copy=True)

datagen=ImageDataGenerator(
  
     featurewise_center=True,
     featurewise_std_normalization=True,
     rotation_range=20,

)
datagen.fit(x_train)

#concatening the old data with augmented data

result_x=np.concatenate((x_train,x_train2),axis=0)
result_y=np.concatenate((y_train,y_train2),axis=0)


In [36]:
cnn_model = create_model()
#opt = SGD(lr = 0.01)
cnn_model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
cnn_model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 32, 32, 64)        1792      
_________________________________________________________________
batch_normalization_13 (Batc (None, 32, 32, 64)        256       
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 32, 32, 64)        36928     
_________________________________________________________________
batch_normalization_14 (Batc (None, 32, 32, 64)        256       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 16, 16, 128)       73856     
__________

In [38]:
from keras.callbacks import EarlyStopping
early_stops = EarlyStopping(patience=5, monitor='val_acc')    #Stop training when a monitored quantity has stopped improving.
                                                              #patience: number of epochs with no improvement after which training will be stopped.
cnn_model.fit(result_x,result_y, batch_size=128, epochs=30, validation_split=0.2, callbacks=[early_stops],shuffle=True)


Train on 80000 samples, validate on 20000 samples
Epoch 1/30
80000/80000 [==============================] - 56s 705us/step - loss: 1.5743 - acc: 0.5238 - val_loss: 1.1875 - val_acc: 0.6162
Epoch 2/30
80000/80000 [==============================] - 54s 679us/step - loss: 1.0498 - acc: 0.6911 - val_loss: 2.1847 - val_acc: 0.3376
Epoch 3/30
80000/80000 [==============================] - 54s 680us/step - loss: 0.9058 - acc: 0.7308 - val_loss: 0.8123 - val_acc: 0.7447
Epoch 4/30
80000/80000 [==============================] - 54s 675us/step - loss: 0.6733 - acc: 0.7960 - val_loss: 0.6498 - val_acc: 0.8033
Epoch 5/30
80000/80000 [==============================] - 54s 674us/step - loss: 0.5856 - acc: 0.8291 - val_loss: 0.5438 - val_acc: 0.8458
Epoch 6/30
80000/80000 [==============================] - 54s 675us/step - loss: 0.5301 - acc: 0.8511 - val_loss: 0.7964 - val_acc: 0.7675
Epoch 7/30
80000/80000 [==============================] - 54s 674us/step - loss: 0.4993 - acc: 0.8668 - val_loss: 0.

In [0]:

cnn_model.optimizer.lr=0.0005

In [0]:
predictions = cnn_model.predict(x_test)